In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display


In [2]:
super = pd.read_csv("Sample - Superstore.csv", encoding='ISO-8859-1')

In [ ]:
super.head(20) # checking the first 20 rows of the data
super.tail(20) # the last 20 rows

In [ ]:
super.info() # checking the overview of the datasets (columns, their data types, and non-null counts)

In [ ]:
super.describe() # summaries for numerical columns

In [6]:
pd.set_option('display.max_columns', None) # I want to see all the columns
pd.set_option('display.max_rows', None) # I want to see all the rows

In [ ]:
super.iloc[9000:9994] # showing the datas from rows 9000 to 9994

In [ ]:
# Step 1: Calculate total sales by region
region_sales = super.groupby('Region')['Sales'].sum().reset_index() # reset_index() turns the index back to a column for easier manipulation

region_sales_sorted = region_sales.sort_values(by='Sales', ascending=False) # sorting the data according to their sales

# Step 2: Sort values to find the highest and lowest sales
highest_sales = region_sales['Sales'].max()
lowest_sales = region_sales['Sales'].min()

# Step 3: Calculate the percentage difference
percentage_difference = ((highest_sales - lowest_sales) / lowest_sales) * 100

# Displaying the result
print(f"The percentage difference between the highest and lowest region sales is {percentage_difference:.2f}%.")

# Step 4: Create a bar chart
plt.figure(figsize=(10,6))
sns.barplot(x='Region', y='Sales', data=region_sales_sorted, hue='Region', palette='viridis')
plt.title('Total Sales by Region')
plt.ylabel('Total Sales ($)')
plt.xlabel('Region')
# I want to show the percentage difference on the visualization
# ha is horizontal allignment and va is for vertical
plt.text(1, highest_sales, f'Percentage difference: {percentage_difference:.2f}%', color='red', fontsize=12, ha='center', va='bottom')

# Display the chart
plt.show()

In [ ]:
# in the case that you want to dive in deeper on the sales of the region
top_region = region_sales_sorted.iloc[0]['Region']  # This extracts the name of the region with the highest sales
top_region_data = super[super['Region'] == top_region]

state_sales = top_region_data.groupby('State')['Sales'].sum().reset_index()
state_sales_sorted = state_sales.sort_values(by='Sales', ascending=False)

# Create a bar plot to visualize sales by state in the top region
plt.figure(figsize=(10,6))
sns.barplot(x='State', y='Sales', data=state_sales_sorted, hue='State', palette='magma')

# Add title and labels
plt.title(f'Sales by States in the Top Region: {top_region}', fontsize=16)
plt.ylabel('Total Sales ($)', fontsize=12)
plt.xlabel('State', fontsize=12)

# Show the plot
plt.xticks(rotation=45)  # Rotate state names for better readability
plt.show()


In [ ]:
# top 5 products according to sales

product_sales = super.groupby('Product Name')[['Sales', 'Quantity', 'Profit']].sum().round(2).reset_index()
product_sales_sorted = product_sales.sort_values(by='Sales', ascending=False)
# getting the top 5 products
top_5_product = product_sales_sorted.head(5)

plt.figure(figure=(10,6))
sns.barplot(x='Sales', y='Product Name', data=top_5_product, hue='Sales', palette='magma')

plt.title(f'Top 5 product by sales', fontsize=16)
plt.yticks(fontsize=8) # adjusting the fontsize of the product names
plt.ylabel('Top Products', fontsize=12)
plt.xlabel('Sales ($)', fontsize=12)
plt.show()

# showing a table that summarizes the product sales, quantity and profit
display(top_5_product[['Product Name', 'Sales', 'Quantity', 'Profit']])

In [ ]:
# getting the monthly sales trend with a 3 month Moving Average

# Making sure that column Order Date is on a date format
super['Order Date'] = pd.to_datetime(super['Order Date'])
# Creating a new column for months
super['Order Month'] = super['Order Date'].dt.to_period('M')
#super['Order Year'] = super['Order Date'].dt.year()

# now calculating the 
monthly_sales = super.groupby('Order Month')[['Sales', 'Profit']].sum().round(2).reset_index()
# converts the the period back to timestamp
monthly_sales['Order Month'] = monthly_sales['Order Month'].dt.to_timestamp()
# getting the sales data trend on a 3 month period
monthly_sales['3-Month MA'] = monthly_sales['Sales'].rolling(window=3).mean()

plt.figure(figsize=(10,6))
plt.title('Monthly Sales Trend', fontsize=16)
sns.lineplot(x='Order Month', y='Sales', data=monthly_sales, label='Monthly Sales', marker='o')
sns.lineplot(x='Order Month', y='3-Month MA', data=monthly_sales, label='3-Month MA', linestyle='--')
plt.ylabel('Sales ($)', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# getting the monthly sales trend for a year
# example year: 2016

# Making sure that column Order Date is on a date format
super['Order Date'] = pd.to_datetime(super['Order Date'])
# Creating a new column for months
super['Order Month'] = super['Order Date'].dt.to_period('M')
super['Order Year'] = super['Order Date'].dt.year

# now calculating the 
yearly_sales = super.groupby('Order Year')[['Sales', 'Profit']].sum().round(2).reset_index()
sales_year_2016 = super[super['Order Year'] == 2016] # filtering the year to 2016
monthly_sales = sales_year_2016.groupby('Order Month')[['Sales', 'Profit']].sum().reset_index()
monthly_sales[['Sales', 'Profit']] = monthly_sales[['Sales', 'Profit']].round(2)
monthly_sales['Order Month'] = monthly_sales['Order Month'].dt.to_timestamp() # converting the order month back to time stamp for plotting

plt.figure(figsize=(10,6))
plt.title('Monthly Sales Trend for year 2016', fontsize=16)
sns.lineplot(x='Order Month', y='Sales', data=monthly_sales)
plt.ylabel('Sales ($)', fontsize=12)
plt.xlabel('Month', fontsize=12)
plt.show()

In [ ]:
# Customer segmentation by Sales

customer_sales = super.groupby('Segment')[['Sales', 'Profit']].sum().round(2).reset_index()

plt.figure(figsize=(10,6))
plt.pie(customer_sales['Sales'], labels=customer_sales['Segment'], autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
plt.axis('equal')
plt.title('Customer Segmentation by Sales', fontsize=16)
plt.show()
display(customer_sales[['Segment', 'Sales', 'Profit']])

In [ ]:
# Sales vs Profit Correlation

category_sales = super.groupby('Category')[['Sales', 'Profit', 'Quantity', 'Discount']].sum().round(2).reset_index()
display(category_sales[['Category', 'Sales', 'Profit', 'Quantity', 'Discount']])

category_sales['Normalized Discount'] = category_sales['Discount'] / category_sales['Discount'].max()

plt.figure(figsize=(10,6))
scatter = plt.scatter(x=category_sales['Sales'],
            y=category_sales['Profit'],
            c=category_sales['Normalized Discount'],
            s=category_sales['Quantity'],
            cmap='coolwarm',
            alpha=0.7)
plt.colorbar(scatter, label='Discount (Normalized)')
plt.title('Sales vs. Profit by Category', fontsize=16)
plt.xlabel('Sales ($)', fontsize=12)
plt.ylabel('Profit ($)', fontsize=12)
# Adding labels in the visualization
for i, category in enumerate(category_sales['Category']):
    plt.text(category_sales['Sales'][i], category_sales['Profit'][i], category, fontsize=9, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Shipping Performance Analysis

super['Order Date'] = pd.to_datetime(super['Order Date'])
super['Ship Date'] = pd.to_datetime(super['Ship Date'])

super['Ship Time (Days)'] = (super['Ship Date'] - super['Order Date']).dt.days
shipping_sales = super.groupby('Ship Time (Days)')[['Sales', 'Profit']].sum().round(2).reset_index()
display(shipping_sales[[ 'Ship Time (Days)', 'Sales', 'Profit']])

plt.figure(figsize=(10,6))
plt.scatter(shipping_sales['Ship Time (Days)'], shipping_sales['Sales'], color='darkblue', label='Sales')
plt.scatter(shipping_sales['Ship Time (Days)'], shipping_sales['Profit'], color='darkred', label='Profit')

plt.title('Shipping Time vs Sales and Profit', fontsize=16)
plt.xlabel('Shipping Time (Days)', fontsize=12)
plt.ylabel('Sales/Profit ($)', fontsize=12)
plt.legend()
plt.tight_layout()
plt.show()